### Steps:
1) import libraries and data
2) build a resampler function of returns (bootstrapping) with numpy and execute with numba
3) build and parallelize a function that applies backtest_bollinger_bands to every generated array, and returns the equity in another set of arrays

#### Imports

In [1]:
#-----------------IMPORTS-----------------#
from main_func import * #we import numba, numpy and the backtest function from here
import pandas as pd #we separately import pandas to read data from a csv file


#-----------------DATA-----------------#
#1min EURGBP data
url = "https://onedrive.live.com/download?resid=4E16D53638D067A9%21339325&authkey=!AHOCBUokDjDiCc8"

#1 second EURGBP data
#url = "https://onedrive.live.com/download?resid=4E16D53638D067A9%21339754&authkey=!ACcJZZPFqOmKVUY"

data_downloaded = pd.read_csv(url, parse_dates = True, index_col=0) #it's a time series data, so I'm setting the index to be the first column
close = data_downloaded["close"].to_numpy() #We transform the close prices into a numpy array

#### Resampler function in Numpy and Numba

In [2]:
#create the function that resamples returns, and returns a matrix of simulated price paths

@njit(fastmath = True) #We use the @njit decorator to compile the function to machine code using the numba library
def bootstrap_returns(close, n_simulations):
    


    #-----------------CALCULATE ACTUAL RETURNS-----------------#
    close_returns = np.zeros(len(close)) #We create an array of zeros with the same length as the close prices,
                                     #so the first return will be zero. We will calculate the returns from the second element;
    for i in range(1, len(close)):
        close_returns[i] = (close[i] - close[i-1])/close[i-1] #We calculate the returns of the close prices



    #-----------------BOOTSTRAPPED RETURNS-----------------#
    simulations = np.zeros((len(close_returns), n_simulations)) #rows = len(close_returns), columns = n_simulations
    for i in range(n_simulations):
        simulations[:,i] = np.random.choice(close_returns, size = len(close_returns), replace = False) #Put in every column the returns of the close prices in a random order

    #-----------------SIMULATED PRICES-----------------#
    prices = np.zeros((len(close_returns), n_simulations))
    prices[0,:] = close[0] #We set the first row of the prices matrix to be the first close price, so they start from the same point

    for i in range(1, len(close)): #from the second row to the last
        prices[i,:] = prices[i-1,:] * (1 + simulations[i,:]) #cumulative returns, every row calculates using the
                                                             #rows from the simulations matrix that contains bootstrapped returns in each column.

    return prices #return the simulated prices matrix

ThousandSimulations = bootstrap_returns(close, 1000) #We create 1000 simulations of the returns

In [9]:
@njit(fastmath = True, parallel=False) #We use the @njit decorator to compile the function to machine code using the numba library
def montecarlo_optimizer(close, n_simulations, window, num_std_devs, fees):
    ThousandSimulations = bootstrap_returns(close, n_simulations) #We create n_simulations simulations of the returns
    montecarlo_equities = np.zeros((len(close), n_simulations)) #We create a matrix of zeros with the same shape as the close prices and n_simulations columns
    for i in range(n_simulations): #for every column
        montecarlo_equities[:,i] = backtest_bollinger_bands(ThousandSimulations[:,i], window=window, num_std_devs= num_std_devs, fees_percentage=fees) #we backtest the bollinger bands strategy with the simulated prices
    
    return montecarlo_equities #return the matrix of equities

In [11]:
from time import time
### Report how many simulations were profitable
number_simulations = 1000

start_time = time()
simulated_equities = montecarlo_optimizer(close, n_simulations=number_simulations, window=120, num_std_devs=10, fees=0) #We create 100 simulations of the equities
end_time = time()
print("Time taken to run the simulations in seconds:", end_time - start_time) #We print the time taken to run the simulations

profitable_simulations = np.sum(simulated_equities[-1,:] > 10000) / number_simulations #We count how many simulations ended with more than 10k
print("profitable simulations in %:", profitable_simulations * 100) #We print the percentage of profitable simulations


#Need to make a function that iterates over different parameters and returns the best ones





#-----------------RESULTS-----------------#
#Time taken to run 10 thousand the simulations in seconds: 739.1393160820007
#profitable simulations in %: 51.23

Time taken to run the simulations in seconds: 44.002654790878296
profitable simulations in %: 50.6
